In [ ]:
%pip install YOLO

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
from ultralytics import YOLO
import cv2
from PIL import Image
import numpy as np
from pathlib import Path

from iris.config.localization_pipeline_config_manager import LocalizationPipelineConfigManager
from iris.config.data_pipeline_config_manager import DataPipelineConfigManager
from iris.data_pipeline.mongodb_manager import MongoDBManager

In [ ]:
# load pre-trained model
detection_model = YOLO("../checkpoints/detect/train/weights/best.pt")

In [ ]:
# Initialize configuration managers
localization_config = LocalizationPipelineConfigManager()
data_config = DataPipelineConfigManager()
shop_config = data_config.shop_configs["pas_normal_studios"]  # Select shop
mongodb_config = data_config.mongodb_config

# Create MongoDB manager
mongodb_manager = MongoDBManager(shop_config, mongodb_config)

In [ ]:
image_metadata = mongodb_manager.get_collection(mongodb_config.image_metadata_collection)
test_images = list(image_metadata.aggregate([{ "$sample": { "size": 1 } }]))

for image_data in test_images:
    image_path = Path(image_data['local_path'])
    # Load image in BGR format
    image = cv2.imread(str(image_path))
    if image is None:
        print(f"Could not load image: {image_path}")
        continue

    # Run prediction (YOLO expects BGR format)
    results = detection_model.predict(
        source=image,  # Pass BGR image directly
        conf=0.3,
        save=False,
        line_thickness=2,
        hide_labels=False
    )

    # Plot results
    plt.figure(figsize=(20,10))
    plt.axis('off')
    annotated_frame = results[0].plot()  # Returns BGR image
    plt.imshow(annotated_frame[...,::-1])  # Convert BGR to RGB using array slicing
    plt.title(f"Predictions for {image_path.name}")
    plt.show()

In [ ]:
results = detection_model.predict(
        source=image,  # Pass BGR image directly
        conf=0.3,
        save=False,
        line_thickness=2,
        hide_labels=False
    )

In [ ]:
results[0].boxes.xywhn

In [ ]:
results[0].boxes.xyxyn